# 텍스트 전처리 파이프라인

<img src='https://raw.githubusercontent.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs/refs/heads/main/cover.png'
     alt="NLP와 LLM 실전 가이드(한빛미디어)"
     style="border: 3px solid gray; box-shadow: 5px 5px 15px rgba(0, 0, 0, 0.3); border-radius: 10px; width: 300px;"   width='300'>


* 저자:  
    - [Lior Gazit](https://www.linkedin.com/in/liorgazit).  
    - [Meysam Ghaffari](https://www.linkedin.com/in/meysam-ghaffari-ph-d-a2553088/).
* 역자:
    - [박조은](https://github.com/corazzon)
* 이 노트북은 다음의 책에서 소개하는 내용입니다.
    - 역서 : NLP와 LLM 실전 가이드(한빛미디어)
    - 원서 : [Mastering NLP from Foundations to LLMs](https://www.amazon.com/dp/1804619183)

**이 노트북의 목적:**  
책의 4장에서 설명했듯이, 텍스트 전처리는 자연어 처리(NLP)에서 가장 기본적인 작업 중 하나입니다.  
이 노트북에서는 다양한 전처리 기능을 단계별로 설명하며, 이를 통합하여 견고한 파이프라인을 구성하는 방법을 보여줍니다.  

**필수 사항:**  
* Colab에서 실행 시, 다음 런타임 노트북 설정을 사용하세요: `Python3, CPU`

colab 실습 : 
https://github.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/Mastering-NLP-from-Foundations-to-LLMs/blob/main/Chapter4_notebooks/Ch4_Preprocessing_Pipeline.ipynb)  


원서 Colab 실습:  
https://github.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs   
<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/blob/liors_branch/Chapter4_notebooks/Ch4_Preprocessing_Pipeline.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

>*```면책사항: 이 노트북에서 다루는 내용과 아이디어는 저자들 개인의 것이며, 저자들의 고용주의 견해나 지적 재산을 대변하지 않습니다.```*

설치:

In [13]:
# 주의사항:
# 아래 코드에서 Python 패키지 불일치로 인한 오류가 발생하는 경우, 새로운 버전이 원인일 수 있습니다.
# 이런 경우, "default_installations"를 False로 설정하여 원래 이미지로 되돌리세요:
default_installations = True
if default_installations:
    !pip install -q num2words autocorrect
else:
    import requests
    text_file_path = "requirements__Ch4_Preprocessing_Pipeline.txt"
    url = "https://raw.githubusercontent.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/main/Chapter4_notebooks/" + text_file_path
    res = requests.get(url)
    with open(text_file_path, "w") as f:
        f.write(res.text)

    !pip install -r requirements__Ch4_Preprocessing_Pipeline.txt

In [14]:
# Imports:
import re
from num2words import num2words
import nltk; nltk.download('punkt'); nltk.download('stopwords'); nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [15]:
# 전처리 함수:
def decode(text):
    """
    입력 문자열에서 주제 줄(subject line)과 본문 텍스트(body text)를
    정규식을 사용해 추출합니다. 추출된 텍스트를 단일 문자열로
    포맷하여 반환합니다.

    입력: str
    출력: str
    """
    text = re.sub("\\n|\\r|\\t|-", " ", text)
    subject_line_search = re.search(r"<SUBJECT LINE>(.*?)<END>", text, flags=re.S)
    body_text_search = re.search(r"<BODY TEXT>(.*?)<END>", text, flags=re.S)

    formated_output = ""
    if subject_line_search:
        formated_output = formated_output + subject_line_search.groups()[0] + ". "
    if body_text_search:
        formated_output = formated_output + body_text_search.groups()[0] + "."
    return formated_output


def digits_to_words(match):
    """
    문자열로 된 숫자를 영어 단어로 변환합니다. 이 함수는 기수(cardinal)와
    서수(ordinal)를 구분합니다.
    예: "2"는 "two"로, "2nd"는 "second"로 변환됩니다.

    입력: str
    출력: str
    """
    suffixes = ['st', 'nd', 'rd', 'th']
    # 이전 작업에 의존하지 않도록 소문자로 변환:
    string = match[0].lower()
    if string[-2:] in suffixes:
        type = 'ordinal'
        string = string[:-2]
    else:
        type = 'cardinal'

    return num2words(string, to=type)


def spelling_correction(text):
    """
    잘못된 철자를 올바른 철자로 교정합니다.

    입력: str
    출력: str
    """
    corrector = Speller()
    spells = [corrector(word) for word in text.split()]
    return " ".join(spells)


def remove_stop_words(text):
    """
    불용어(stopwords)를 제거합니다.

    입력: str
    출력: str
    """
    stopwords_set = set(stopwords.words('english'))
    return " ".join([word for word in text.split() if word not in stopwords_set])


def stemming(text):
    """
    각 단어에 대해 어간 추출(stemming)을 수행합니다.

    입력: str
    출력: str
    """
    stemmer = PorterStemmer()
    return " ".join([stemmer.stem(word) for word in text.split()])


def lemmatizing(text):
    """
    각 단어에 대해 표제어 추출(lemmatization)을 수행합니다.

    입력: str
    출력: str
    """
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [16]:
# 전처리 파이프라인:
def preprocessing(input_text, printing=False):
    """
    이 함수는 텍스트 전처리를 위한 전체 파이프라인을 나타냅니다.

    코드 설계 참고: "output" 변수를 업데이트하면서 진행하는 방식은
    새로운 변수 이름을 생성하지 않으므로 작업 순서를 변경하거나
    작업을 추가/제거하는 것이 용이합니다.

    입력: str
    출력: str
    """
    output = input_text

    # 디코딩/인코딩 제거:
    output = decode(output)
    print("\n디코딩/인코딩 제거:\n        ", output)

    # 소문자로 변환:
    output = output.lower()
    print("\n소문자로 변환:\n        ", output)

    # 숫자를 단어로 변환:
    # 다음 정규 표현식은 연속된 숫자와 서수 접미사를 잠정적으로 포함한 패턴을 찾습니다:
    output = re.sub(r'\d+(st)?(nd)?(rd)?(th)?', digits_to_words, output, flags=re.IGNORECASE)
    print("\n숫자를 단어로 변환:\n        ", output)

    # 구두점 및 기타 특수 문자 제거:
    output = re.sub('[^ A-Za-z0-9]+', '', output)
    print("\n구두점 및 기타 특수 문자 제거:\n        ", output)

    # 철자 교정:
    output = spelling_correction(output)
    print("\n철자 교정:\n        ", output)

    # 불용어 제거:
    output = remove_stop_words(output)
    print("\n불용어 제거:\n        ", output)

    # 어간 추출:
    output = stemming(output)
    print("\n어간 추출:\n        ", output)

    # 표제어 추출:
    output = lemmatizing(output)
    print("\n표제어 추출:\n        ", output)

    return output


In [17]:
# Applying preprocessing:
raw_text_input = """
"<SUBJECT LINE> Employees details<END><BODY TEXT>Attached are 2 files,\n1st one is pairoll, 2nd is healtcare!<END>"
"""
print(f"This is the input raw text:\n{raw_text_input}")

print(f"\n----------------------------\nThis is the preprocessed text:\n        {preprocessing(raw_text_input, printing=True)}")

This is the input raw text:

"<SUBJECT LINE> Employees details<END><BODY TEXT>Attached are 2 files,
1st one is pairoll, 2nd is healtcare!<END>"


디코딩/인코딩 제거:
          Employees details. Attached are 2 files, 1st one is pairoll, 2nd is healtcare!.

소문자로 변환:
          employees details. attached are 2 files, 1st one is pairoll, 2nd is healtcare!.

숫자를 단어로 변환:
          employees details. attached are two files, first one is pairoll, second is healtcare!.

구두점 및 기타 특수 문자 제거:
          employees details attached are two files first one is pairoll second is healtcare

철자 교정:
         employees details attached are two files first one is payroll second is healthcare

불용어 제거:
         employees details attached two files first one payroll second healthcare

어간 추출:
         employe detail attach two file first one payrol second healthcar

표제어 추출:
         employe detail attach two file first one payrol second healthcar

----------------------------
This is the preprocessed text:
        emplo